In [0]:
%run /Workspace/Users/vishal.krishnan@neudesic.com/Silver_to_Gold/utils

In [0]:
# fact_campaign_cost
meta_cost = df_meta.select("campaign_id", "date", "platform_name", "ad_spend", "clicks", "conversions", "likes", "comments", "shares") \
    .withColumn("total_engagement", col("likes") + col("comments") + col("shares")) \
    .withColumn("platform_id", expr("case when lower(platform_name) = 'facebook' then 101 when lower(platform_name) = 'instagram' then 102 else null end")) \
    .withColumnRenamed("date", "cost_date")

youtube_cost = df_youtube.select("campaign_id", "date", "estimated_revenue", "clicks", "conversions", "likes", "comments", "shares") \
    .withColumn("total_engagement", col("likes") + col("comments") + col("shares")) \
    .withColumn("platform_id", lit(103)) \
    .withColumnRenamed("date", "cost_date") \
    .withColumnRenamed("estimated_revenue", "ad_spend")
fact_cost = meta_cost.select("campaign_id", "platform_id", "cost_date", "ad_spend", "clicks", "conversions", "total_engagement") \
    .unionByName(youtube_cost.select("campaign_id", "platform_id", "cost_date", "ad_spend", "clicks", "conversions", "total_engagement"))

upsert_to_delta(fact_cost, "fact_campaign_cost", ["campaign_id", "platform_id", "cost_date"],
                zorder_cols=["campaign_id", "platform_id", "cost_date"])